# IS 4487 Lab 8

## Outline

*   Deal with nulls
*   Standardize/transform attributes
*   Convert attributes to factors (categories)
*   Encode attributes

<a href="https://colab.research.google.com/github/Stan-Pugsley/is_4487_base/blob/main/Labs/Scripts/lab8_megatelco_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Megatelco Data Dictionary

 DEMOGRAPHIC VARIABLES:
 - College - has the customer attended some college (one, zero)
 - Income - annual income of customer
 - House - estimated price of the customer's home (if applicable)

 USAGE VARIABLES:
 - Data Overage Mb - Average number of megabytes that the customer used in excess of the plan limit (over last 12 months)
 - Data Leftover Mb - Average number of megabytes that the customer use was below the plan limit (over last 12 months)
 - Data Mb Used - Average number of megabytes used per month (over last 12 months)
 - Text Message Count - Average number of texts per month (over last 12 months)
 - Over 15 Minute Calls Per Month - Average number of calls over 15 minutes in duration per month (over last 12 months)
 - Average Call Duration- Average call duration (over last 12 months)

PHONE VARIABLES:
 - Operating System - Current operating system of phone
 - Handset Price - Retail price of the phone used by the customer

ATTITUDINAL VARIABLES:
 - Reported Satisfaction - Survey response to "How satisfied are you with your current phone plan?" (high, med, low)
 - Reported Usage Level - Survey response to "How much do your use your phone?" (high, med, low)
 - Considering Change of Plan - Survey response to "Are you currently planning to change companies when your contract expires?" (high, med, low)

OTHER VARIABLES
 - Leave - Did this customer churn with the last contract expiration? (LEAVE, STAY)
 - ID - Customer identifier

##Load Libraries

In this class we will be using
- Pandas
- Scikitlearn
- Matplotlib


In [5]:
import pandas as pd
import matplotlib as mpl
import numpy as np

from sklearn.tree import DecisionTreeClassifier, export_graphviz # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import tree


## Import Data into Dataframe

➡️ Assignment Tasks
- Import data from the Megatelco dataset into a dataframe (in GitHub go to Labs > DataSets)
- Describe or profile the dataframe

In [7]:
df = pd.read_csv('megatelco_leave_survey.csv')
df.describe()

,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,house,handset_price,over_15mins_calls_per_month,average_call_duration,id
count,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,1.499900e+04,14999.000000,14996.000000,14999.000000,14999.000000
mean,241989.140476,153.522502,37.522035,4202.624442,135.622041,8.771616e+05,795.146810,10.568685,9.731515,11859.749517
std,109603.370138,113.018111,28.046409,2202.763649,48.831061,2.870316e+05,1235.785429,8.402100,5.195150,6812.842446
min,-65000.000000,0.000000,0.000000,400.000000,52.000000,-4.630000e+02,215.000000,0.000000,1.000000,2.000000
25%,147822.000000,54.000000,12.000000,2294.000000,93.000000,6.444405e+05,498.000000,3.000000,5.000000,6137.000000
50%,241656.000000,151.000000,35.000000,4223.000000,135.000000,8.764130e+05,778.000000,9.000000,10.000000,11760.000000
75%,336456.500000,242.000000,62.000000,6080.000000,178.000000,1.098834e+06,1064.000000,17.000000,14.000000,17396.000000
max,432000.000000,380.000000,89.000000,8000.000000,220.000000,1.456389e+06,125000.000000,35.000000,19.000000,25354.000000


## Clean up the data
Did you notice anything unusual about the "house" amounts?

How about the handset price and income?

Clean up the data in a  new datafram named "df_clean"


In [8]:
#delete rows with outlier data; put it in a new dataframe
df_clean = df[(df['house'] > 0) & (df['income'] > 0) & (df['handset_price'] < 1000)]

#delete any rows with missing values in the clean dataframe
df_clean = df_clean.dropna()

df_clean.describe()

,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,house,handset_price,over_15mins_calls_per_month,average_call_duration,id
count,10412.000000,10412.000000,10412.000000,10412.000000,10412.000000,1.041200e+04,10412.000000,10412.000000,10412.000000,10412.000000
mean,242068.987706,153.847003,37.472436,4195.399827,135.500000,8.758348e+05,607.473492,10.571072,9.747503,11839.793988
std,109041.788157,113.013727,28.015291,2205.189091,48.916131,2.868363e+05,225.506388,8.444673,5.180381,6805.159729
min,52491.000000,0.000000,0.000000,400.000000,52.000000,3.202380e+05,215.000000,0.000000,1.000000,2.000000
25%,148548.000000,55.000000,12.000000,2290.750000,93.000000,6.447640e+05,413.000000,3.000000,5.000000,6070.750000
50%,241663.000000,151.000000,35.000000,4206.500000,135.000000,8.730785e+05,608.000000,9.000000,10.000000,11743.000000
75%,335660.500000,242.000000,62.000000,6079.250000,178.000000,1.098298e+06,802.000000,17.000000,14.000000,17330.500000
max,431916.000000,380.000000,89.000000,8000.000000,220.000000,1.456389e+06,999.000000,35.000000,19.000000,25354.000000


# Standardize attributes

- Convert the "college" and "considering_change_of_plan" attributes to 0 or 1
- Convert non-numeric attributes to numeric scale of 1-3


In [9]:
#Get distinct values
df_clean['college'].unique()

array(['one', 'zero'], dtype=object)

In [10]:
df_clean['reported_satisfaction'] .unique()

array(['low', 'high', 'avg'], dtype=object)

In [11]:
df_clean['reported_usage_level'].unique()

array(['low', 'high', 'avg'], dtype=object)

In [12]:
df_clean['considering_change_of_plan'].unique()

array(['yes', 'no', 'maybe'], dtype=object)

In [13]:
df_clean.loc[df_clean['college'] == 'one', 'college'] = "1"
df_clean.loc[df_clean['college'] == 'zero', 'college'] = "0"
df_clean.loc[df_clean['reported_satisfaction'] == 'low', 'reported_satisfaction'] = "1"
df_clean.loc[df_clean['reported_satisfaction'] == 'avg', 'reported_satisfaction'] = "2"
df_clean.loc[df_clean['reported_satisfaction'] == 'high', 'reported_satisfaction'] = "3"
df_clean.loc[df_clean['reported_usage_level'] == 'low', 'reported_usage_level'] = "1"
df_clean.loc[df_clean['reported_usage_level'] == 'avg', 'reported_usage_level'] = "2"
df_clean.loc[df_clean['reported_usage_level'] == 'high', 'reported_usage_level'] = "3"
df_clean.loc[df_clean['considering_change_of_plan'] == 'yes', 'considering_change_of_plan'] = "1"
df_clean.loc[df_clean['considering_change_of_plan'] == 'no', 'considering_change_of_plan'] = "0"
df_clean.loc[df_clean['considering_change_of_plan'] == 'maybe', 'considering_change_of_plan'] = "0.5"

df_clean['college'] = df_clean['college'].astype('int')
df_clean['reported_satisfaction'] = df_clean['reported_satisfaction'].astype('int')
df_clean['reported_usage_level'] = df_clean['reported_usage_level'].astype('int')
df_clean['considering_change_of_plan'] = df_clean['considering_change_of_plan'].astype('float')

df_clean.head(10)

,college,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,house,handset_price,over_15mins_calls_per_month,average_call_duration,reported_satisfaction,reported_usage_level,considering_change_of_plan,leave,id,operating_system
0,1,403137,70,0,6605,199,841317,653,5.0,8,1,1,1.0,LEAVE,8183,Android
7,1,131230,0,0,4489,89,613744,365,5.0,10,1,3,1.0,STAY,18170,Android
8,1,193667,0,0,1023,76,949829,347,1.0,8,3,1,0.0,LEAVE,3201,Android
9,1,419573,0,74,4631,209,818027,640,0.0,2,1,1,1.0,LEAVE,12612,Android
10,0,262085,222,17,530,95,781944,692,10.0,1,3,3,1.0,LEAVE,3266,Android
11,1,310841,74,14,7025,180,979451,835,3.0,5,3,1,0.0,STAY,13155,IOS
12,0,398292,0,0,3681,93,878343,248,0.0,9,2,2,0.0,LEAVE,1491,Android
13,0,192245,96,40,4806,139,1141088,476,4.0,2,1,3,1.0,LEAVE,13418,Android
16,1,236152,0,21,3881,189,1364832,745,13.0,6,3,1,1.0,STAY,3335,Android
17,1,422168,0,10,794,213,909127,874,1.0,4,3,1,1.0,STAY,80,IOS


# Convert attributes to factors

- Leave
- College
- Reported satisfaction
- Reported usage level
- Considering change of plan

In [20]:
#Method #1
#df_clean['leave'] = pd.Categorical(df_clean['leave'])

#Method #2
df_clean['leave'] = df_clean['leave'].astype('category')
df_clean['college'] = df_clean['college'].astype('category')
df_clean['reported_satisfaction'] = df_clean['reported_satisfaction'].astype('category')
df_clean['reported_usage_level'] = df_clean['reported_usage_level'].astype('category')
df_clean['considering_change_of_plan'] = df_clean['considering_change_of_plan'].astype('category')

df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10412 entries, 0 to 14998
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   college                      10412 non-null  category
 1   income                       10412 non-null  int64   
 2   data_overage_mb              10412 non-null  int64   
 3   data_leftover_mb             10412 non-null  int64   
 4   data_mb_used                 10412 non-null  int64   
 5   text_message_count           10412 non-null  int64   
 6   house                        10412 non-null  int64   
 7   handset_price                10412 non-null  int64   
 8   over_15mins_calls_per_month  10412 non-null  float64 
 9   average_call_duration        10412 non-null  int64   
 10  reported_satisfaction        10412 non-null  category
 11  reported_usage_level         10412 non-null  category
 12  considering_change_of_plan   10412 non-null  category
 13  leave 

## Calculate distribution of the outcome  
What is the proportion of people who churned?

Why should we care about this proportion?

1. An important step in EDA is to understand the distribution of the target variable.

2. The majority class in the target variable will serve as an important benchmark for model performance. If we used what we'll call a "majority class classifier"---this consists in always predicting the majority class, which in this case is `STAY`---we would be correct 1 - .49 or 51% of the time.  Another way of saying this is that majority class classifier in the MegaTelCo case would result in accuracy of .51.  

Accuracy is defined as the proportion of correctly predicted labels. It is a commonly used error metric for evaluating classifier performance.

Think about why a majority class model in this case would have an accuracy of .51.

Whatever later model we develop should have better accuracy than this performance benchmark.

In [15]:
#Add new field with binary value for leave
df_clean['leave_flag'] = df_clean['leave'].str.replace('STAY','0')
df_clean['leave_flag'] = df_clean['leave_flag'].str.replace('LEAVE','1')

#Convert to integer
df_clean['leave_flag'] = df_clean['leave_flag'].astype('int')

#Find the mean value
df_clean['leave_flag'].mean()


0.4990395697272378

## Prepare Data

➡️ Assignment Tasks
- Create a new variable called "Income_Level".   Use the income variable to put customers in to groups from $0-$100K, $101-200K, etc.
- Create a new variable called "High_Risk".  If the satisfaction level is low and the customer is considering a plan change, then the use "1" for high_risk; otherwise use zero.  Make this a categorical variable.  

In [19]:
#income level
df_clean['Income_Level'] = pd.cut(x=df['income'], bins=[0, 100000, 200000, 300000, 400000, 500000],
                     labels=['$0-100K', '$101-200K', '$201-300K', '$301-400K', '$401-500K'])


,Income_Level
0,$401-500K
7,$101-200K
8,$101-200K
9,$401-500K
10,$201-300K
...,...
14992,$301-400K
14993,$201-300K
14994,$301-400K
14997,$201-300K


In [47]:
df_clean['reported_satisfaction'].value_counts()

,count
reported_satisfaction,
1,7561
3,2347
2,504


In [46]:
#high risk
#df_clean['High_Risk'] = df_clean['reported_satisfaction'].replace(2,0)
#df_clean['High_Risk'] = df_clean['High_Risk'].replace(3,0)
df_clean['High_Risk'] = df_clean.loc[df_clean['reported_satisfaction'] == 2, 'High_Risk'] = 0
df_clean['High_Risk'] = df_clean.loc[df_clean['reported_satisfaction'] == 3, 'High_Risk'] = 0
#df_clean['High_Risk'] = df_clean.loc[df_clean['reported_satisfaction'] == 1, 'High_Risk'] = 1

df_clean['High_Risk'] = df_clean['High_Risk'].astype('category')
df_clean['High_Risk'].value_counts()

,count
High_Risk,
0,10412


## Visualize Relationships

➡️ Assignment Tasks
- Create a bar chart, histogram, or scatter for at least three variables, showing their relationship with the target variable

In [ ]:
#chart 1

In [ ]:
#chart 2

In [ ]:
#chart 3